<a href="https://colab.research.google.com/github/tekajurt/Bert-example/blob/main/BERT_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#importamos las librerias
import os
import pandas as pd
import re

In [6]:
#Vamos a instalar unas libreria, especificamente la libreria transformers de "huggingface"
!pip install transformers

In [7]:
#importamos la libreria de transformers
import transformers
#y multiples clasificadores
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup

In [8]:
#Importamos la libreria para hacer el split train/test
from sklearn.model_selection import train_test_split

In [9]:
# Importamos las librerias de pytorch
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [10]:
# Bajamos los datos
!curl -O http://srodriguez.me/Datasets/imdb.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.7M  100 25.7M    0     0  5280k      0  0:00:04  0:00:04 --:--:-- 5969k


In [11]:
#Los descomprimimos
!unzip imdb.zip

Archive:  imdb.zip
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace IMDB Dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [12]:
#Leemos el DataFrame
df = pd.read_csv("IMDB Dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [13]:
#mostramos los primeros 5
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
#Muestreamos 10.000 reviews al azar
df = df.sample(10000)
df.reset_index(drop=True,inplace=True)

In [15]:
# Transformamos el output en valores numericos
df['sentiment'] = df['sentiment'].map({'positive':1,'negative':0})

In [16]:
df.head()

,review,sentiment
0,It is very possible that I simply didn't give ...,0
1,The Movie Freddy's dead the final nightmare is...,1
2,OK I had higher hopes for this Carnosaur movie...,0
3,For all the cast and crew who worked on this e...,0
4,I love watching steven seagal movies not becau...,0


In [17]:
#Mostramos la primer review
df.review[0]

"It is very possible that I simply didn't give the movie a fair enough chance because it was so immediately unappealing to me (something similar happened with Triplets of Belleville), but I really should have caught on when I put the film on and my roommate, an exchange student from Japan, immediately started laughing at the movie, saying that it sounded dumb. Now, I don't agree that it is dumb, the animation is very simple but clearly very skilled. It's like classic animation with added layers that add another element of realism to them while remaining strictly in the realm of the surreal at the same time.<br /><br />But the subject matter is entirely unappealing to me. It has an interesting message about stinginess and greed, but it is wrapped in such an unpleasant package that it is almost not worth learning, especially because you already knew it anyway. The sound effects while the man is loudly gobbling cherries and cherry pits, for example, are indeed repulsive.<br /><br />There 

In [18]:
#Obtenemos set de train, validacion y testing
temp_df, test_df = train_test_split(df,test_size=0.3,random_state=42)
train_df, val_df = train_test_split(temp_df,test_size=0.1,random_state=42)

train_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [19]:
val_df

,review,sentiment
0,It is way beyond me how this script was ever s...,0
1,"my girlfriend, as we walk in the cold London e...",1
2,I am curious of what rifle Beckett was using i...,1
3,John Cassavetes is on the run from the law. He...,1
4,Original Claymation Rudolph: Pretty good. Orig...,0
...,...,...
695,Where do I begin? Let me say that -- after hav...,1
696,Would someone tell shaq to stick to what he is...,0
697,Very disappointing 7th chapter of this slowly ...,0
698,"Odd one should be able to stumble into ""Classe...",1


In [20]:
lengths = train_df.review.apply(lambda x: len(x.split(" ")))
lengths.max()

1737

In [23]:
#Cargamos un modelo de bert y su tokenizador
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [24]:
#Hagamos una prueba de como bert tokeniza una oración
tokenizer.tokenize("Hi, this is a test to see how Bert tokenizer processes a sentence")

['hi',
 ',',
 'this',
 'is',
 'a',
 'test',
 'to',
 'see',
 'how',
 'bert',
 'token',
 '##izer',
 'processes',
 'a',
 'sentence']

In [25]:
#El vocabulario no es necesario ajustarlo, ya viene pre-entrenado y po lo mismo, 
#La transformación a indices númericos se utiliza con el metodo encode plus
tokenizer.encode_plus("Hi, this is a test to see how Bert tokenizer processes a sentence")

{'input_ids': [101, 7632, 1010, 2023, 2003, 1037, 3231, 2000, 2156, 2129, 14324, 19204, 17629, 6194, 1037, 6251, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [49]:
#Vamos a generar un CustomDataset (una formar de cargar datos en batches)
#Para pytorch, nos va a facilitar el entrenamiento
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.sentence = dataframe.review.values
        self.targets = self.data.sentiment.values
        self.max_len = max_len
    def __len__(self):
        return len(self.sentence)
    def __getitem__(self, index):
        sentence = str(self.sentence[index])
        sentence = " ".join(sentence.split())
        inputs = self.tokenizer.encode_plus(
            sentence,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True

        )
        ids = inputs['input_ids'] #Ids del vocabulario
        mask = inputs['attention_mask'] #Mascaras para definir donde la atencion deberia ver
        token_type_ids = inputs["token_type_ids"] #requerido pero no utilizado
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [50]:
#Generamos nuestra conexion a la GPU
from torch import cuda
device = 'cuda:0' if cuda.is_available() else 'cpu'

In [51]:
device

'cuda:0'

In [52]:
# Definimos constantes para la ejecucion de nuestro codigo
MAX_LEN = 200 # 2470
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-05

In [53]:
#Transformamos los df en datasets
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
val_set = CustomDataset(val_df, tokenizer, MAX_LEN)
test_set = CustomDataset(test_df, tokenizer, MAX_LEN)

In [54]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

#Luego generamos los dataloaders de pytorch

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(test_set, **test_params)

In [55]:
#Vamos a definir una clase de BERT modificada
class BERTClass(torch.nn.Module):
    def __init__(self,b_model):
        super(BERTClass, self).__init__()
        self.l1 = b_model # Definimos que la primera capa sea el modelo de bert que recibe por parametro
        self.l2 = torch.nn.Dropout(0.1) # A la salida de bert, vamos a aplicar un dropout de 0.1
        self.l3 = torch.nn.Linear(768, 1) #Finalmente vamos a definir una capa densa (combinación lineal) de la dimension de bert a nuestra neurona de salida
        self.out_activation = torch.nn.Sigmoid() # función de activación (que no vamos a utilizar)
    
    def forward(self, ids, mask, token_type_ids):
        #Hacemos el forward pass por BERT, dropout y la capa densa
        _, output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids) 
        output_2 = self.l2(output_1)
        output = self.l3(output_2)
        #output = self.out_activation(output)
        return output

#Generamos un modelo de BERTClass
model = BERTClass(bert_model)
#llevamos el modelo a la GPU
model.to(device)

BERTClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

In [56]:
#Definimos nuestra funcion de perdida
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [57]:
#El optimizador
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [58]:
from tqdm.notebook import tqdm

In [59]:
#Definimos nuestro loop de entrenamiento
def train(epoch):
    print('asd')
    model.train()
    loss_acum=0
    for iters , data in tqdm(enumerate(training_loader, 0), total = len(training_loader)):
        #print(len(data['ids'][0]))
        #print(len(data['mask'][0]))
        #print(len(data['token_type_ids'][0]))
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        #print(len(ids),len(mask),len(token_type_ids))
        #print(ids,mask,token_type_ids,targets)
        outputs = model(ids, mask, token_type_ids)
        outputs = outputs.squeeze(1)
        #print(outputs,targets)
        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        loss_acum += loss.item()
        if iters%5000==0:
            print(outputs,targets)
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(iters)
    return loss_acum/iters

In [60]:
#Definimos nuestro loop de validacion/testing
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0),total = len(testing_loader)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)

            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).squeeze(1).cpu().detach().numpy())
    
    return np.array(fin_outputs), np.array(fin_targets)

In [61]:
#iteramos por nuestras epocas
for epoch in range(EPOCHS):
    epoch_loss = train(epoch)
    print(f'epoch loss: {epoch_loss}')

asd


tensor([ 0.1691, -0.6475, -0.2777, -0.2919, -0.6222, -0.5438, -0.6121, -0.0792],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 1., 1., 1., 1., 1., 1., 0.], device='cuda:0')
Epoch: 0, Loss:  0.9117749333381653

787
epoch loss: 0.3257305964832271
asd


tensor([-3.9512,  3.4260, -3.9089,  1.1882,  4.3464,  2.4628, -0.6605, -0.3042],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 1., 0., 1., 1., 1., 0., 0.], device='cuda:0')
Epoch: 1, Loss:  0.1750771403312683

787
epoch loss: 0.1793596010437422
asd


tensor([-3.9137,  3.8495,  2.5342, -4.3796,  4.0393,  3.0948, -1.0193, -4.5479],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([0., 1., 1., 0., 1., 1., 0., 0.], device='cuda:0')
Epoch: 2, Loss:  0.06375274807214737

787
epoch loss: 0.0940893067081318
asd


tensor([ 5.0369,  5.4372, -5.2583,  3.8501, -5.3286,  3.8041,  4.4854,  2.1621],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([1., 1., 0., 1., 0., 1., 1., 1.], device='cuda:0')
Epoch: 3, Loss:  0.023008592426776886

787
epoch loss: 0.05669945060793817
asd


tensor([ 4.9178, -6.3336, -6.0565, -6.2546, -2.3455, -6.2782,  4.7002, -5.6147],
       device='cuda:0', grad_fn=<SqueezeBackward1>) tensor([1., 0., 0., 0., 0., 0., 1., 0.], device='cuda:0')
Epoch: 4, Loss:  0.014921903610229492

787
epoch loss: 0.04419893267998777


In [ ]:
from sklearn import metrics
import numpy as np

#Corremos un loop de testing
for epoch in range(1):
    outputs, targets = validation(epoch)

In [ ]:
#Calculamos los valores binarios y luego obtenemos las metricas de desempeño
outputs_bin = (outputs >= 0.5)
accuracy = metrics.accuracy_score(targets, outputs_bin)
f1_score_micro = metrics.f1_score(targets, outputs_bin, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs_bin, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
print(metrics.classification_report(targets, outputs_bin))